<img style="float: right" src="images/surfsara.png">
<br/>
<hr style="clear: both" />

# Introduction to RDDs - Apache Spark

This notebook provides an introduction to Apache Spark RDD API using PySpark. Press Shift-Enter to execute the code. You can use code completion by using tab.

During the exercises you may want to refer to [The PySpark documentation](https://spark.apache.org/docs/latest/api/python/pyspark.html) for more information on possible transformations and actions. We will provide links to the documentation when we introduce methods on RDDs.

## The SparkContext

The SparkContext contains all the information about the way Spark is set up. When running on a cluster, the SparkContext contains the address of the cluster and will make sure operations on RDDs will be executed there. In the cell below, we create a [`SparkContext`](https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.SparkContext) using `local mode`. This means that Spark will run locally, not on a cluster. It will offer some form of parallelism by making use of the various cores it has available.

Note, that Spark is best used in `cluster mode` where it will run on many machines simultaneously. `Local mode` is only meant for training or testing purposes. However, Spark works quite well in local mode and can be quite powerful. In order to run locally developed code on a cluster, the only thing that needs to be changed is the `SparkContext` and paths to in- and output files.

Even when working in `local mode` it is important to think of an RDD as a data structure that is distributed over many machines on a cluster, and is not available locally. The machine that contains the `SparkContext` is called the *driver*. The SparkContext will communicate with the cluster manager to make sure that the operations on RDDs will run on the cluster in the form of *workers*. It is important to realize that the driver is a separate entity from the nodes in the cluster. You can consider the notebook as being the driver.

## Install

In [1]:
%pip install nltk  # nltk is a natural language processing toolkit  and  has features like tokenization, stemming, etc.

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Initialize Spark
from pyspark import SparkContext, SparkConf
import nltk
nltk.download('punkt') # Download the punkt tokenizer  which is used to split the text into sentences
nltk.download('averaged_perceptron_tagger')  # Download the averaged_perceptron_tagger which is used to tag the words with their parts of speech (POS) i.e noun, verb, adjective, etc.

if not 'sc' in globals(): # This 'trick' makes sure the SparkContext sc is initialized exactly once
    conf = SparkConf().setMaster('local[*]')  # Spark will use all cores (*) available
    sc = SparkContext(conf=conf) # Initialize SparkContext sc with the above configuration conf 

[nltk_data] Downloading package punkt to /home/amrit/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/amrit/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


22/10/10 17:44:45 WARN Utils: Your hostname, AMRIT resolves to a loopback address: 127.0.1.1; using 172.27.198.74 instead (on interface eth0)
22/10/10 17:44:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/10 17:44:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Creating an RDD

There are three ways to create an RDD: by transforming an existing one, by reading in data, or by creating an RDD based on a local data structure. We show this last option below.

A Python list containing some words is used to create an RDD by calling [`parallelize`](https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.SparkContext.parallelize), a method of `SparkContext`. This list is very small and will not benefit from the parallelism of Spark. 

We then print the number of records in the RDD, by calling the `count()` method.

In [3]:
words_list = ['Dog', 'Cat', 'Rabbit', 'Hare', 'Deer', 'Gull', 'Woodpecker', 'Mole']
words_rdd = sc.parallelize(words_list)
print(words_rdd.count())  

8


## Map transformation 
There are two kinds of operations on RDDs: transformations and actions. Transformations take as input an RDD and produce as output another RDD (you cannot change an existing RDD, they are immutable). Computation of transformations is deferred until an *action* is executed. An action does not return an RDD, but instead returns data to the driver (for example in the form of a Python list), or writes data to disk or a database.

This *laziness* of executing transformations allows Spark to optimize computations. Only when the user wants real output, the framework will start to compute.

One of the most used transformations is [`map`](https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.map). This is very similar to the `Map` in MapReduce. The Spark version of `Map` is a method called `map` defined on an RDD, and takes as input a single function. This function will be applied to each element in the RDD, and Spark will put the result of the application in the output RDD.

First, we present a simple Python function that takes a single word as argument and returns the word with an 's' added to it. In the next step we will use this function in a map transformation of the `words_rdd`.

Take a look at the function definition below and execute it.

In [4]:
def make_plural(word):
    return word + 's'

# Let's see if it works

print(make_plural('cat'))

cats


Next, we want to use the `make_plural` function as input for the `map` transformation on `words_rdd`.
The action [collect()](https://spark.apache.org/docs/latest/api/python/pyspark.html?highlight=collect#pyspark.RDD.collect) transfers the content of the RDD to the driver. The result of `collect()` will then be available to our local environment in Python. It is not an RDD but a Python list!

Note, that a large RDD may be scattered over many machines. In such a case calling `collect()` may not be a good idea, since it can take quite some time to retrieve all RDD parts.

## Exercise 1
In the cell below enter the name of the function that map should apply to each element of the RDD in order to end up with an RDD of words in plural form.

In [5]:
plural_rdd = words_rdd.map(make_plural) # Map the make_plural function to each element of the words_rdd

print(plural_rdd.collect()) 

['Dogs', 'Cats', 'Rabbits', 'Hares', 'Deers', 'Gulls', 'Woodpeckers', 'Moles']


## Using lambda functions
We can achieve the same functionality by using lambda functions. In this case we define `make_plural` not using `def` as we did above, but as an anonymous function that we define inside `map` directly. This is the main benefit of using lambda functions: all our processing logic is directly visible in the transformations we're applying.

## Exercise 2
Provide a lambda function in the cell below, that will pluralize all elements in the RDD.

In [6]:
lambda_plural_rdd = words_rdd.map(lambda word: word + 's') # Map the lambda function to each element of the words_rdd

#lambda symtax is: lambda <argument>: <expression>

print(lambda_plural_rdd.collect())

['Dogs', 'Cats', 'Rabbits', 'Hares', 'Deers', 'Gulls', 'Woodpeckers', 'Moles']


## Exercise 3
Another transformation is [filter()](https://spark.apache.org/docs/latest/api/python/pyspark.html?highlight=filter#pyspark.RDD.filter). It takes as argument a predicate function (a function that is evaluated to true or false), and applies the predicate to all elements of the RDD. Only elements that are evaluated to true by the filter function, will be passed on to the output RDD.

Use the [filter()](https://spark.apache.org/docs/latest/api/python/pyspark.html?highlight=filter#pyspark.RDD.filter) method of RDD to keep only words with a length larger than three. Use a lambda function to write a predicate that does this. Next, [count()](https://spark.apache.org/docs/latest/api/python/pyspark.html?highlight=count#pyspark.RDD.count) the number of words. 

Like `collect`, [count()](https://spark.apache.org/docs/latest/api/python/pyspark.html?highlight=count#pyspark.RDD.count) is an action. Remember that actions trigger Sparks computations. Transformations are evaluated lazily and their computation is deferred until an action is called.

There should be 6 words that pass the filter. 

In [7]:
filtered_rdd = words_rdd.filter(lambda word: len(word) > 3) # Filter the words_rdd to keep only words with length > 3
print(filtered_rdd.collect())
filtered_rdd.count() 


['Rabbit', 'Hare', 'Deer', 'Gull', 'Woodpecker', 'Mole']


6

## Exercise 4

Let's do another `map` transformation on words_rdd. For each word in word_rdd determine its length, again using a lambda function.

In [8]:
word_lengths =  words_rdd.map(lambda word: len(word)).collect()
print(words_rdd.collect())
print(word_lengths)

['Dog', 'Cat', 'Rabbit', 'Hare', 'Deer', 'Gull', 'Woodpecker', 'Mole']
[3, 3, 6, 4, 4, 4, 10, 4]


## FlatMap transformation
Sometimes, the result of a `map` operation is a list of elements rather than a single element. Consider the following example, where we have a list of sentences, and we split each sentence:

In [9]:
sentences = sc.parallelize([
    'this is a sentence',
    'and this is another one'
])
sentences_rdd = sentences.map(str.split)
sentences_rdd.collect()

[['this', 'is', 'a', 'sentence'], ['and', 'this', 'is', 'another', 'one']]

Each element in the RDD returned by `map` is a list of words. Consequently, the result of `collect` is a list of lists, each list containing the sentences' words. Hence, a `count` of this RDD will return two:

In [10]:
sentences_rdd.count()

2

If we want to count the number of words instead, or work directly with the words, we will need to _flatten_ the list of lists into a single list. To do so, we will substitute `flatMap` for `map`. Like `map`, `flatMap` will apply the supplied function to each element in the RDD. In addition to `map`, though, it will _flatten_ the result of the operation such that a list of lists becomes a list:

In [11]:
sentences.flatMap(str.split).collect()

['this', 'is', 'a', 'sentence', 'and', 'this', 'is', 'another', 'one']

## Pair RDDs
Pair RDDs are very important within the Spark RDD API. Each element of a Pair RDD is a pair (or tuple) `(x,y)` where `x` is interpreted as being the key and `y` as the value. Spark offers quite a number of `...byKey` and `...byValues` methods that operate on pair RDDs. As we will see, these methods can be used to define functions per key, very similar to Hadoop's MapReduce.

Keys can be of any *hashable* type, which means all primitive types (numbers, strings, etc.), tuples, **but not lists or dictionaries**. Values can be of any type.

Below we define a Python string variable called `sonnet`. It is assigned Shakespeare's first sonnet in the form of a single line of text. The character `\` is used to let Python ignore the new line character. 

Execute the cell, otherwise the variable is not declared and assigned a value.

In [12]:
sonnet = "From fairest creatures we desire increase, \
That thereby beauty\'s rose might never die, \
But as the riper should by time decease, \
His tender heir might bear his memory: \
But thou contracted to thine own bright eyes, \
Feed'st thy light's flame with self-substantial fuel, \
Making a famine where abundance lies, \
Thy self thy foe, to thy sweet self too cruel: \
Thou that art now the world's fresh ornament, \
And only herald to the gaudy spring, \
Within thine own bud buriest thy content, \
And, tender churl, mak'st waste in niggarding: \
Pity the world, or else this glutton be, \
To eat the world\'s due, by the grave and thee."

## Python magic

From this text we first remove punctuation. The next cell is just Python. You may want to skip this if your focus is just on Spark, but don't forget to execute the cell.

`maketrans()` is a Python method on strings that very efficiently can make character substitutions. Below we use it to remove all punctuation characters. The curly braces indicate a dictionary, and the expression within it, is called a comprehension. The result is a dictionary of key-value pairs, called table, where the key is a punctuation character and the value is `None`. When making substitutions by means of `translate` this table then removes all the entries that have a `None` value.

In [13]:
import string

# The following line creates a translation table
table = str.maketrans({key: None for key in string.punctuation})

# Do a sample translation
s = "string. With. Punctuation?"
print(s.translate(table)) 

string With Punctuation


## Parallelizing the text

In the next cell a lot is happening in one line. The text above is first translated - which in this case means that each punctuation character is removed. Then on the result, the `lower()` method is applied. (This is a Python method on strings.) This puts a string in lowercase letters. Then this result is `split()`, meaning that the text is split in individual words. (Also a Python method on strings). This results in a list of words, all lowercase, with no punctuation. This is input to the `parallelize()` method which turns it into an RDD.

*Calling consecutive methods by using dot-notation is called chaining. It is possible of course to execute these steps individually, but chaining can be very convenient, especially in Spark. Consider the individual steps: first parallelize the text, then map the resulting RDD to remove the punctuation, then map the resulting RDD to lowercase the text and then map the resulting RDD of that step to split the data... Doing this instead by chaing methods safes a lot of typing.* 

To show just the 5 first elements, we use Spark's [`take()`](https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.take) action. This limits the amount of data that is sent to the driver.

In [14]:
text_rdd = sc.parallelize(sonnet.translate(table).lower().split()) 
text_rdd.take(20)

['from',
 'fairest',
 'creatures',
 'we',
 'desire',
 'increase',
 'that',
 'thereby',
 'beautys',
 'rose',
 'might',
 'never',
 'die',
 'but',
 'as',
 'the',
 'riper',
 'should',
 'by',
 'time']

## Exercise 5

What would happen if we wouldn't split the text but directly transform it into an RDD? Try this in the next cell (omit `translate` and `lower` as well).

Try to predict what will happen. Remember that a string in Python is very similar to a list. 

(For a list called `mylist` the first element is given by `mylist[0]`. Similarly `mystring[0]` will return the first character of the string `mystring`.)

In [15]:
another_rdd = sc.parallelize(sonnet.split())   
another_rdd.take(20)  # punctuation  like '(apporstorpe) ,(comma) are visible here  

['From',
 'fairest',
 'creatures',
 'we',
 'desire',
 'increase,',
 'That',
 'thereby',
 "beauty's",
 'rose',
 'might',
 'never',
 'die,',
 'But',
 'as',
 'the',
 'riper',
 'should',
 'by',
 'time']

## Exercise 6
We are going to count the words in `text_rdd`. As a first step, transform every word in `text_rdd` into a tuple `(<word>, 1)`. Use a lambda function.

In [16]:
pair_rdd = text_rdd.map(lambda word: (word,1)).reduceByKey(lambda a, b: a + b)
pair_rdd.take(20)


[('we', 1),
 ('increase', 1),
 ('thereby', 1),
 ('as', 1),
 ('riper', 1),
 ('tender', 2),
 ('bear', 1),
 ('selfsubstantial', 1),
 ('fuel', 1),
 ('abundance', 1),
 ('ornament', 1),
 ('herald', 1),
 ('that', 2),
 ('might', 2),
 ('by', 2),
 ('contracted', 1),
 ('to', 4),
 ('lights', 1),
 ('foe', 1),
 ('too', 1)]

## Exercise 7
There is an *action* called [countByKey](https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.countByKey) that performs the counting and returns it as a Python dictionary.
Use it below to see the counts.

In [17]:
word_counts=text_rdd.map(lambda word: (word,1)).countByKey()

# Below is some Python code that will nicely print the word_counts dictionary 

sorted_word_counts = sorted(word_counts.items(), key=lambda x: -x[1])  # Sort the word_counts dictionary by value in descending order 

for word, count in sorted_word_counts[:5]:
    print(word, count)

the 6
thy 5
to 4
and 3
that 2


##  reduceByKey
`countByKey` is an _action_, returning the word counts as a dictionary instead of an RDD. When using `countByKey` with a large number of counts, the dictionary that is sent back to the driver may not fit in memory.

If we want to count words and keep the result into an RDD we have to use the [reduceByKey](https://spark.apache.org/docs/latest/api/python/pyspark.html?highlight=reducebykey#pyspark.RDD.reduceByKey) *transformation*.

This transformation works almost exactly like Reduce in Hadoop's MapReduce. It expects the RDD to consist of key-value pairs and it will perform a reduce operation *per key*.

As input [reduceByKey](https://spark.apache.org/docs/latest/api/python/pyspark.html?highlight=reducebykey#pyspark.RDD.reduceByKey) takes a *two-argument function* that will be applied on the values when they are grouped by key.

## Exercise 8
Create a lambda function that does the counting and forms the input for `reduceByKey`.

In [18]:
# Note that reduceByKey takes in a function that accepts two values and returns a single value
# The function that is input to reduceByKey only works on the values. Spark will execute this function per key

word_counts = pair_rdd.reduceByKey(lambda a, b: a + b) # In this case we are adding the values of the same key together to get the total count of each word.  
print(word_counts.collect())

[('we', 1), ('increase', 1), ('thereby', 1), ('as', 1), ('riper', 1), ('tender', 2), ('bear', 1), ('selfsubstantial', 1), ('fuel', 1), ('abundance', 1), ('ornament', 1), ('herald', 1), ('that', 2), ('might', 2), ('by', 2), ('contracted', 1), ('to', 4), ('lights', 1), ('foe', 1), ('too', 1), ('cruel', 1), ('and', 3), ('buriest', 1), ('or', 1), ('be', 1), ('never', 1), ('but', 2), ('thou', 2), ('own', 2), ('eyes', 1), ('thy', 5), ('flame', 1), ('making', 1), ('sweet', 1), ('in', 1), ('this', 1), ('grave', 1), ('from', 1), ('the', 6), ('decease', 1), ('with', 1), ('lies', 1), ('art', 1), ('fresh', 1), ('gaudy', 1), ('spring', 1), ('within', 1), ('niggarding', 1), ('eat', 1), ('fairest', 1), ('creatures', 1), ('beautys', 1), ('rose', 1), ('die', 1), ('his', 2), ('heir', 1), ('thine', 2), ('bright', 1), ('famine', 1), ('where', 1), ('self', 2), ('now', 1), ('worlds', 2), ('only', 1), ('churl', 1), ('pity', 1), ('world', 1), ('due', 1), ('desire', 1), ('should', 1), ('time', 1), ('memory', 1

Instead of using `collect` we can use [takeOrdered](https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.takeOrdered) to see the most frequent words first.

Below we show 10 elements from the RDD. The elements are pairs and we sort them by the second element (denoted by `x[1]` in the lambda function. The minus indicates descending order.

In [19]:
word_counts.takeOrdered(10, lambda x: -x[1])

[('the', 6),
 ('thy', 5),
 ('to', 4),
 ('and', 3),
 ('tender', 2),
 ('that', 2),
 ('might', 2),
 ('by', 2),
 ('but', 2),
 ('thou', 2)]

## Analysing tweets
So far we have created our RDDs with our own strings and lists. Typically, though, you will read data from file or a database.

In the remainder of the notebook, we will analyse Dutch tweets that we load from file using [sc.textFile](https://spark.apache.org/docs/latest/api/python/pyspark.html?highlight=textfile#pyspark.SparkContext.textFile). Each tweet is on a single line in the file, formatted a JSON dictionary. `sc.textFile` will read the each line as text, and we will need to convert this text to JSON later. We'll do this in exercise 10. The result of `sc.textFile` will be an RDD of strings, each string containing a single line (a tweet) of the file.

The file we load the tweets from is a local file in our case. Often when using Spark files reside on a distributed file system like HDFS. When creating the RDD Spark may distribute the data over many machines.

First, let's look at the first line of the data file we are going to use. We use a simple Unix command here (no Python) to view the first line of a file that resides on local disk. Notice, that this is a single tweet in JSON. 

In [20]:
# Unix bash command called head.
# The ! announces a Unix command is coming to Jupyter

# !head -1 Data/tweets.json

## Exercise 9
Below the call to `sc.textFile` is made. There are also empty lines in the text file (i.e. their length is equal to 0). Provide a lambda function to the subsequent `filter` call to remove these empty lines.

Lastly, print out the first tweet in the RDD by making use of the [take](https://spark.apache.org/docs/latest/api/python/pyspark.html?highlight=take#pyspark.RDD.take) action.

In [21]:
# Note that we need to use the backslash character to have multiline statements in Python

tweets_rdd = sc \
    .textFile('Data/tweets.json') \
    .filter( lambda line: len(line) >0 )

# print(tweets_rdd.take(1))


## Conversion to JSON
Next, we are going to convert the tweets into dictionaries. For this purpose we import the Python `json` library. In Python a string `s` is converted to a dictionary by calling `json.loads(s)`.

After conversion, each tweet will be a dictionary where each key-value pair is an attribute of the tweet. Some attributes have sub-attributes, such as the ones contained under the `user` key.

## Exercise 10
Transform each tweet in the `tweets_rdd` to dictionary, and print the first tweet.

In [28]:
import json

json_tweets_rdd = tweets_rdd.flatMap(json.loads)

# Print out the first element (tweet) of the resulting RDD. The last line will format the tweet for you.
parsed_tweet = json_tweets_rdd.take(1)[0]  # [0][0] is used to get the first tweet 
print(json.dumps(parsed_tweet, indent=4, sort_keys=True)) 

{
    "contributors": null,
    "coordinates": null,
    "created_at": "Wed Oct 21 22:41:26 +0000 2020",
    "entities": {
        "hashtags": [],
        "symbols": [],
        "urls": [
            {
                "display_url": "twitter.com/i/web/status/1\u2026",
                "expanded_url": "https://twitter.com/i/web/status/1319046163778179072",
                "indices": [
                    117,
                    140
                ],
                "url": "https://t.co/SxT1MJO1nE"
            }
        ],
        "user_mentions": []
    },
    "favorite_count": 18497,
    "favorited": false,
    "geo": null,
    "id": 1319046163778179072,
    "id_str": "1319046163778179072",
    "in_reply_to_screen_name": null,
    "in_reply_to_status_id": null,
    "in_reply_to_status_id_str": null,
    "in_reply_to_user_id": null,
    "in_reply_to_user_id_str": null,
    "is_quote_status": false,
    "lang": "en",
    "place": null,
    "possibly_sensitive": false,
    "retweet_count

In [29]:
json_tweets_rdd.collect()  #testing 

[{'created_at': 'Wed Oct 21 22:41:26 +0000 2020',
  'id': 1319046163778179072,
  'id_str': '1319046163778179072',
  'text': 'Exclusive: Forgotten by Obama-Biden Auto Bailout, Delphi Workers Refuse to Forget What Was Taken from Them (Part On… https://t.co/SxT1MJO1nE',
  'truncated': True,
  'entities': {'hashtags': [],
   'symbols': [],
   'user_mentions': [],
   'urls': [{'url': 'https://t.co/SxT1MJO1nE',
     'expanded_url': 'https://twitter.com/i/web/status/1319046163778179072',
     'display_url': 'twitter.com/i/web/status/1…',
     'indices': [117, 140]}]},
  'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
  'in_reply_to_status_id': None,
  'in_reply_to_status_id_str': None,
  'in_reply_to_user_id': None,
  'in_reply_to_user_id_str': None,
  'in_reply_to_screen_name': None,
  'user': {'id': 25073877,
   'id_str': '25073877',
   'name': 'Donald J. Trump',
   'screen_name': 'realDonaldTrump',
   'location': 'Washington, DC',
   'descrip

## Accessing fields in tweets
In the cell below some fields from the tweets are selected. Notice that the input `x` for the lambda function is a dictionary containing the tweet. The result of the lambda function (defined after `:`) is a list with values of the selected fields from the tweet.

You should be able to figure out how to select information from a tweet, after looking at this example.

In [36]:
json_tweets_rdd_text = json_tweets_rdd.map(
    lambda x:[
        x['lang'],
        x['entities']['hashtags'],
        x['user']['name'],
        x['user']['screen_name'],
        x['user']['followers_count'],
        x['user']['description']
    ] 
    
)
json_tweets_rdd_text.take(1)

[['en',
  [],
  'Donald J. Trump',
  'realDonaldTrump',
  87331221,
  '45th President of the United States of America🇺🇸']]

## Selecting Text
We will work with the text of the tweets in the next few cells.

## Exercise 11
From the `json_tweets_rdd` select **only** the tweet text. (Do not put the text in a list, like we did above with the fields we selected there).

In [40]:
# TODO: Replace <FILL IN> with appropriate code

tweet_text_rdd = json_tweets_rdd.map( lambda x: x['text'] )

tweet_text_rdd.take(5)

['Exclusive: Forgotten by Obama-Biden Auto Bailout, Delphi Workers Refuse to Forget What Was Taken from Them (Part On… https://t.co/SxT1MJO1nE',
 'Multiple Pro-Trump Demonstrations Planned in Cincinnati Area https://t.co/jjMeB9Hba0 via @BreitbartNews',
 'Pennsylvania Trump Voters Show Passion: ‘He‘s a Man that Wants to Do It All for America‘ https://t.co/1oYXtzAyhT via @BreitbartNews',
 'WSJ Editorial Board: Joe Biden Must Answer Questions About Hunter Biden and China https://t.co/pF2M9i9OSu via @BreitbartNews',
 'See you soon North Carolina! https://t.co/MWfyg4ux69 https://t.co/hfbrBTgu15']